In [3]:
# imports

import os
import numpy as np
from tinygrad import Tensor, nn
import matplotlib.pyplot as plt

# setting tinygrad device to gpu since METAL is broken on older macs
os.environ['GPU'] = '1'

In [15]:
integer = Tensor([2])
print(integer.realize(), integer.ndim, integer.shape)

decimal = Tensor([3.14159])
print(decimal.realize())

<Tensor <UOp GPU (1,) int (<Ops.BUFFER: 7>, <buf real:True device:GPU size:1 dtype:dtypes.int>)> on GPU with grad None> 1 (1,)
<Tensor <UOp GPU (1,) float (<Ops.BUFFER: 7>, <buf real:True device:GPU size:1 dtype:dtypes.float>)> on GPU with grad None>


In [23]:
fibonacci = Tensor([1,1,2,3,5,8])
print(fibonacci.ndim, fibonacci.shape)

count_to_100 = Tensor.arange(start=0, stop=100, step=1)
print(count_to_100.ndim, count_to_100.shape)

1 (6,)
1 (100,)


In [25]:
### Defining higher-order Tensors ###

# define a 2-d tensor
matrix = Tensor([[1,2], [3,4]])
assert matrix.ndim == 2

# define a 4-d tensor
# Use Tensor.zeros to initialize a 4-d Tensor of zeros with size 10 x 3 x 256 x 256.
#   You can think of this as 10 images where each image is RGB 256 x 256.
images = Tensor.zeros(10,3,256,256)
assert images.ndim == 4
assert images.shape == (10,3,256,256)
print(f"images is a {images.ndim}-d Tensor with shape: {images.shape}")

images is a 4-d Tensor with shape: (10, 3, 256, 256)


In [32]:
row_vector = matrix[1]
column_vector = matrix[:, 1]
scalar = matrix[0, 1]

print(f"`row_vector`: {row_vector.numpy()}")
print(f"`column_vector`: {column_vector.numpy()}")
print(f"`scalar`: {scalar.numpy()}")

`row_vector`: [3 4]
`column_vector`: [2 4]
`scalar`: 2


In [34]:
# Create the nodes in the graph and initialize values
a = Tensor(15)
b = Tensor(61)

# Add them!
c1 = Tensor.add(a, b)
c2 = a + b  # tinygrad overrides the "+" operation so that it is able to act on Tensors
print(f"c1: {c1.numpy()}")
print(f"c2: {c2.numpy()}")

c1: 76
c2: 76


In [41]:
### Defining Tensor computations ###

# Construct a simple computation function
def func(a, b):
    #Define the operation for c, d, e
    a = Tensor(a)
    b = Tensor(b)
    c = Tensor.add(a, b)
    d = b - 1
    e = Tensor.mul(c, d)
    return e

In [49]:
# Consider example values for a,b
a, b = 1.5, 2.5
# Execute the computation
e_out = func(a, b)
print(f"e_out: {e_out}")
print(f"e_out: {e_out.realize()}")
print(f"e_out: {e_out.numpy()}")

e_out: <Tensor <UOp GPU () float (<Ops.MUL: 57>, None)> on GPU with grad None>
e_out: <Tensor <UOp GPU () float (<Ops.CONST: 82>, None)> on GPU with grad None>
e_out: 6.0


In [52]:
### Defining a dense layer ###

# num_inputs: number of input nodes
# num_outputs: number of output nodes
# x: input to the layer

class OurDenseLayer:
    def __init__(self, num_inputs, num_outputs):
        # Define and initialize parameters: a weight matrix W and bias b
        # Note that the parameter initialize is random!
        self.W = Tensor.randn(num_inputs, num_outputs)
        self.bias = Tensor.randn(num_outputs)

    def __call__(self, x):
        #define the operation for z (hint: use .matmul() method)
        z = x.matmul(self.W) + self.bias

        #define the operation for out (hint: use .sigmoid())
        y = z.sigmoid()
        return y

In [54]:
# Define a layer and test the output!
num_inputs = 2
num_outputs = 3
layer = OurDenseLayer(num_inputs, num_outputs)
x_input = Tensor([[1, 2.]])
y = layer(x_input)

print(f"input shape: {x_input.shape}")
print(f"output shape: {y.shape}")
print(f"output result: {y.numpy()}")

input shape: (1, 2)
output shape: (1, 3)
output result: [[0.9301118  0.45047274 0.66546106]]


In [139]:
### Defining a neural network using the tinygrad .sequential method ###

# define the number of inputs and outputs
n_input_nodes = 2
n_output_nodes = 3

# Define the model
# Use the .sequential meth to define a neural network with a
# single linear (dense!) layer, followed by non-linearity to compute z
class Model:
    def __init__(self):
        self.lin = nn.Linear(n_input_nodes, n_output_nodes) # linear dense layer

    def __call__(self, x):
        return x.sequential([
            lambda x: self.lin(x),
            lambda x: x.sigmoid() # non-linear activation function
        ])

In [148]:
# Test the model with example input
x_input = Tensor([[1, 2.]])
model = Model()
y = model(x_input)

print(f"model weights: {model.lin.weight.numpy()}")
print(f"input shape: {x_input.shape}")
print(f"output shape: {y.shape}")
print(f"output result: {y.numpy()}")

model weights: [[-0.48608315  0.4123966 ]
 [-0.615962    0.39451793]
 [ 0.02119346 -0.0608265 ]]
input shape: (1, 2)
output shape: (1, 3)
output result: [[0.6872106  0.45883477 0.52659845]]
